In [1]:
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from optuna.integration import LightGBMPruningCallback


KeyboardInterrupt: 

In [ ]:
    
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission_df = pd.read_csv('sample_submission.csv')
print("Данные успешно загружены.")

Данные успешно загружены.


In [ ]:

features = [col for col in train_df.columns if col not in ['idx', 'type']]
X = train_df[features].copy()
y = train_df['type']

if 'quality_flag' in X.columns:
    le_quality = LabelEncoder()
    mode_quality = X['quality_flag'].mode()[0]
    X['quality_flag'] = X['quality_flag'].fillna(mode_quality)
    test_df['quality_flag'] = test_df['quality_flag'].fillna(mode_quality)
    all_quality_values = pd.concat([X['quality_flag'], test_df['quality_flag']]).unique()
    le_quality.fit(all_quality_values)
    X['quality_flag'] = le_quality.transform(X['quality_flag'])
    test_df['quality_flag'] = le_quality.transform(test_df['quality_flag'])
    X['quality_flag'] = X['quality_flag'].astype('category')
    test_df['quality_flag'] = test_df['quality_flag'].astype('category')

le_type = LabelEncoder()
y_encoded = le_type.fit_transform(y)

In [ ]:
def objective(trial, x_data, y_data, label_encoder):
    boosting_type = trial.suggest_categorical('boosting_type', ['gbdt', 'goss'])
    params = {
        'device': 'gpu',
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_class': len(label_encoder.classes_),
        'boosting_type': boosting_type,
        'n_estimators': trial.suggest_int('n_estimators', 200, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 5, 16),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'class_weight': 'balanced',
        'random_state': 42,
        'verbose': -1,
        'n_jobs': -1,
    }
    if boosting_type == 'gbdt':
        params['bagging_fraction'] = trial.suggest_float('bagging_fraction', 0.5, 0.95)
        params['bagging_freq'] = trial.suggest_int('bagging_freq', 1, 7)
        params['min_child_weight'] = trial.suggest_float('min_child_weight', 1e-5, 1e-1, log=True)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    for i, (train_index, val_index) in enumerate(skf.split(x_data, y_data)):
        X_train_fold, X_val_fold = x_data.iloc[train_index], x_data.iloc[val_index]
        y_train_fold, y_val_fold = y_data[train_index], y_data[val_index]
        model = lgb.LGBMClassifier(**params)
        callbacks = [lgb.early_stopping(25, verbose=False)]
        if i == 0:
            pruning_callback = LightGBMPruningCallback(trial, 'multi_logloss')
            callbacks.append(pruning_callback)
        model.fit(X_train_fold, y_train_fold,
                  eval_set=[(X_val_fold, y_val_fold)],
                  eval_metric='multi_logloss',
                  callbacks=callbacks)
        score = model.best_score_['valid_0']['multi_logloss']
        scores.append(score)
    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(lambda trial: objective(trial, X, y_encoded, le_type), n_trials=80, show_progress_bar=True)



print(f"Лучший Macro F1 Score: {study.best_value:.4f}")
print("Лучшие гиперпараметры:", study.best_params)


best_params = study.best_params
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X, y_encoded)


[I 2025-09-22 23:21:17,893] A new study created in memory with name: no-name-b990d85e-b6b7-4e13-8e3d-f4512b592c5a
  0%|          | 0/80 [00:00<?, ?it/s]

[W 2025-09-22 23:21:17,898] Trial 0 failed with parameters: {'boosting_type': 'gbdt', 'n_estimators': 900, 'learning_rate': 0.07895940302298536, 'num_leaves': 198, 'max_depth': 14, 'min_child_samples': 21, 'feature_fraction': 0.8113950692332795, 'lambda_l1': 0.12578035310911562, 'lambda_l2': 1.990351560943239, 'bagging_fraction': 0.8527931991102757, 'bagging_freq': 1, 'min_child_weight': 0.0014140184890782382} because of the following error: NameError("name 'StratifiedKFold' is not defined").
Traceback (most recent call last):
  File "c:\Users\akaze\Documents\VS Code\NTO\nto_neri\.venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\akaze\AppData\Local\Temp\ipykernel_4836\3340940652.py", line 2, in <lambda>
    study.optimize(lambda trial: objective(trial, X, y_encoded, le_type), n_trials=80, show_progress_bar=True)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

NameError: name 'StratifiedKFold' is not defined

In [ ]:
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X, y_encoded)

,boosting_type,'gbdt'
,num_leaves,3857
,max_depth,10
,learning_rate,0.12999767733969828
,n_estimators,581
,subsample_for_bin,200000
,objective,'multiclass'
,class_weight,'balanced'
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,1221


In [ ]:
X_test = test_df[features]
test_predictions_encoded = final_model.predict(X_test)
predicted_class_names = le_type.inverse_transform(test_predictions_encoded)

id_column_name = sample_submission_df.columns[0]
submission_df = pd.DataFrame({id_column_name: test_df['idx']})

for class_name in le_type.classes_:
    submission_df[class_name] = 0

for i, class_name in enumerate(predicted_class_names):
    submission_df.loc[i, class_name] = 1
    
submission_df = submission_df[sample_submission_df.columns]

submission_df.to_csv('submission_tuned.csv', index=False)

топ

0.973467824363582


